In [3]:
from transformers import pipeline
import gradio as gr

pipeline = pipeline(task="text-classification", model="jonmanly/custServiceClassifier")

def classify(input_txt):
    predictions = pipeline(input_txt)
    return predictions

gradio_app = gr.Interface(
    classify,
    inputs=gr.Text(label="Say what you want here to the Customer Service Classifier", ),
    outputs=gr.Text(label="Classification and Probability Score"),
    title="Customer Service Classifier Demo",
)

if __name__ == "__main__":
    gradio_app.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
